In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

In [3]:
pd.set_option('display.max_columns', 500)

In [5]:
train = pd.read_csv('train.csv')
train.head()

c:\users\lincoln.thiago.neves\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_

In [6]:
train_sample_v01 = train.head()
train_sample_v01.to_csv('train_sample_v01.csv', sep=';' ,index=False)

In [7]:
train.shape

(110938, 190)

In [8]:
train.target.value_counts()

home    48113
away    35174
draw    27651
Name: target, dtype: int64

In [9]:
train['target_perc'] = (1/train.id.count())*100
train.groupby(['target']).agg({'id':'count','target_perc':'sum'})

,id,target_perc
target,,
away,35174,31.705998
draw,27651,24.924733
home,48113,43.369269


In [24]:
train['home_team_history_match_date_1_ano'] = pd.DatetimeIndex(train['home_team_history_match_date_1']).year.astype('str').str[:4]

In [25]:
train['home_team_history_match_date_1_ano'].value_counts()

2020    70075
2021    29830
2019     9874
nan      1159
Name: home_team_history_match_date_1_ano, dtype: int64

In [32]:
#train['home_team_history_match_date_1_mes'] = pd.DatetimeIndex(train['home_team_history_match_date_1']).month

In [31]:
#train['home_team_history_match_date_1_mes'].value_counts()

In [30]:
#train['home_team_history_match_date_1_ano'] = pd.DatetimeIndex(train['home_team_history_match_date_1']).year.astype('str')
#train['home_team_history_match_date_1_mes'] = pd.DatetimeIndex(train['home_team_history_match_date_1']).month.astype('str')
#train['home_team_history_match_date_1_ano_mes'] = train['home_team_history_match_date_1_ano']+train['home_team_history_match_date_1_mes']
#train['home_team_history_match_date_1_ano_mes'].value_counts()

In [42]:
train['home_team_history_is_play_home_perc'] = (train['home_team_history_is_play_home_1']+train['home_team_history_is_play_home_2']+train['home_team_history_is_play_home_3']+train['home_team_history_is_play_home_4']+train['home_team_history_is_play_home_5']+train['home_team_history_is_play_home_6']+train['home_team_history_is_play_home_7']+train['home_team_history_is_play_home_8']+train['home_team_history_is_play_home_9']+train['home_team_history_is_play_home_10']) / 10
train['away_team_history_is_play_home_perc'] = (train['away_team_history_is_play_home_1']+train['away_team_history_is_play_home_2']+train['away_team_history_is_play_home_3']+train['away_team_history_is_play_home_4']+train['away_team_history_is_play_home_5']+train['away_team_history_is_play_home_6']+train['away_team_history_is_play_home_7']+train['away_team_history_is_play_home_8']+train['away_team_history_is_play_home_9']+train['away_team_history_is_play_home_10']) / 10

In [43]:
train['home_team_history_is_cup_perc'] = (train['home_team_history_is_cup_1']+train['home_team_history_is_cup_2']+train['home_team_history_is_cup_3']+train['home_team_history_is_cup_4']+train['home_team_history_is_cup_5']+train['home_team_history_is_cup_6']+train['home_team_history_is_cup_7']+train['home_team_history_is_cup_8']+train['home_team_history_is_cup_9']+train['home_team_history_is_cup_10']) / 10
train['away_team_history_is_cup_perc'] = (train['away_team_history_is_cup_1']+train['away_team_history_is_cup_2']+train['away_team_history_is_cup_3']+train['away_team_history_is_cup_4']+train['away_team_history_is_cup_5']+train['away_team_history_is_cup_6']+train['away_team_history_is_cup_7']+train['away_team_history_is_cup_8']+train['away_team_history_is_cup_9']+train['away_team_history_is_cup_10']) / 10

In [48]:
train['home_team_history_goal_1_ind'] = train['home_team_history_goal_1'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_2_ind'] = train['home_team_history_goal_2'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_3_ind'] = train['home_team_history_goal_3'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_4_ind'] = train['home_team_history_goal_4'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_5_ind'] = train['home_team_history_goal_5'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_6_ind'] = train['home_team_history_goal_6'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_7_ind'] = train['home_team_history_goal_7'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_8_ind'] = train['home_team_history_goal_8'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_9_ind'] = train['home_team_history_goal_9'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_goal_10_ind'] = train['home_team_history_goal_10'].apply(lambda x: 1 if x > 0 else 0)

train['home_team_history_goal_tot_perc'] = (train['home_team_history_goal_1_ind']+train['home_team_history_goal_2_ind']+train['home_team_history_goal_3_ind']+train['home_team_history_goal_4_ind']+train['home_team_history_goal_5_ind']+train['home_team_history_goal_6_ind']+train['home_team_history_goal_7_ind']+train['home_team_history_goal_8_ind']+train['home_team_history_goal_9_ind']+train['home_team_history_goal_10_ind'])



train['away_team_history_goal_1_ind'] = train['away_team_history_goal_1'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_2_ind'] = train['away_team_history_goal_2'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_3_ind'] = train['away_team_history_goal_3'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_4_ind'] = train['away_team_history_goal_4'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_5_ind'] = train['away_team_history_goal_5'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_6_ind'] = train['away_team_history_goal_6'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_7_ind'] = train['away_team_history_goal_7'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_8_ind'] = train['away_team_history_goal_8'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_9_ind'] = train['away_team_history_goal_9'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_goal_10_ind'] = train['away_team_history_goal_10'].apply(lambda x: 1 if x > 0 else 0)

train['away_team_history_goal_tot_perc'] = (train['away_team_history_goal_1_ind']+train['away_team_history_goal_2_ind']+train['away_team_history_goal_3_ind']+train['away_team_history_goal_4_ind']+train['away_team_history_goal_5_ind']+train['away_team_history_goal_6_ind']+train['away_team_history_goal_7_ind']+train['away_team_history_goal_8_ind']+train['away_team_history_goal_9_ind']+train['away_team_history_goal_10_ind'])


train['home_team_history_goal_tot'] = (train['home_team_history_goal_1']+train['home_team_history_goal_2']+train['home_team_history_goal_3']+train['home_team_history_goal_4']+train['home_team_history_goal_5']+train['home_team_history_goal_6']+train['home_team_history_goal_7']+train['home_team_history_goal_8']+train['home_team_history_goal_9']+train['home_team_history_goal_10'])
train['away_team_history_goal_tot'] = (train['away_team_history_goal_1']+train['away_team_history_goal_2']+train['away_team_history_goal_3']+train['away_team_history_goal_4']+train['away_team_history_goal_5']+train['away_team_history_goal_6']+train['away_team_history_goal_7']+train['away_team_history_goal_8']+train['away_team_history_goal_9']+train['away_team_history_goal_10'])

In [49]:
train['home_team_history_opponent_goal_1_ind'] = train['home_team_history_opponent_goal_1'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_2_ind'] = train['home_team_history_opponent_goal_2'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_3_ind'] = train['home_team_history_opponent_goal_3'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_4_ind'] = train['home_team_history_opponent_goal_4'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_5_ind'] = train['home_team_history_opponent_goal_5'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_6_ind'] = train['home_team_history_opponent_goal_6'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_7_ind'] = train['home_team_history_opponent_goal_7'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_8_ind'] = train['home_team_history_opponent_goal_8'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_9_ind'] = train['home_team_history_opponent_goal_9'].apply(lambda x: 1 if x > 0 else 0)
train['home_team_history_opponent_goal_10_ind'] = train['home_team_history_opponent_goal_10'].apply(lambda x: 1 if x > 0 else 0)

train['home_team_history_opponent_goal_tot_perc'] = (train['home_team_history_opponent_goal_1_ind']+train['home_team_history_opponent_goal_2_ind']+train['home_team_history_opponent_goal_3_ind']+train['home_team_history_opponent_goal_4_ind']+train['home_team_history_opponent_goal_5_ind']+train['home_team_history_opponent_goal_6_ind']+train['home_team_history_opponent_goal_7_ind']+train['home_team_history_opponent_goal_8_ind']+train['home_team_history_opponent_goal_9_ind']+train['home_team_history_opponent_goal_10_ind'])



train['away_team_history_opponent_goal_1_ind'] = train['away_team_history_opponent_goal_1'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_2_ind'] = train['away_team_history_opponent_goal_2'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_3_ind'] = train['away_team_history_opponent_goal_3'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_4_ind'] = train['away_team_history_opponent_goal_4'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_5_ind'] = train['away_team_history_opponent_goal_5'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_6_ind'] = train['away_team_history_opponent_goal_6'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_7_ind'] = train['away_team_history_opponent_goal_7'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_8_ind'] = train['away_team_history_opponent_goal_8'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_9_ind'] = train['away_team_history_opponent_goal_9'].apply(lambda x: 1 if x > 0 else 0)
train['away_team_history_opponent_goal_10_ind'] = train['away_team_history_opponent_goal_10'].apply(lambda x: 1 if x > 0 else 0)

train['away_team_history_opponent_goal_tot_perc'] = (train['away_team_history_opponent_goal_1_ind']+train['away_team_history_opponent_goal_2_ind']+train['away_team_history_opponent_goal_3_ind']+train['away_team_history_opponent_goal_4_ind']+train['away_team_history_opponent_goal_5_ind']+train['away_team_history_opponent_goal_6_ind']+train['away_team_history_opponent_goal_7_ind']+train['away_team_history_opponent_goal_8_ind']+train['away_team_history_opponent_goal_9_ind']+train['away_team_history_opponent_goal_10_ind'])


train['home_team_history_opponent_goal_tot'] = (train['home_team_history_opponent_goal_1']+train['home_team_history_opponent_goal_2']+train['home_team_history_opponent_goal_3']+train['home_team_history_opponent_goal_4']+train['home_team_history_opponent_goal_5']+train['home_team_history_opponent_goal_6']+train['home_team_history_opponent_goal_7']+train['home_team_history_opponent_goal_8']+train['home_team_history_opponent_goal_9']+train['home_team_history_opponent_goal_10'])
train['away_team_history_opponent_goal_tot'] = (train['away_team_history_opponent_goal_1']+train['away_team_history_opponent_goal_2']+train['away_team_history_opponent_goal_3']+train['away_team_history_opponent_goal_4']+train['away_team_history_opponent_goal_5']+train['away_team_history_opponent_goal_6']+train['away_team_history_opponent_goal_7']+train['away_team_history_opponent_goal_8']+train['away_team_history_opponent_goal_9']+train['away_team_history_opponent_goal_10'])

In [57]:
train['home_team_history_rating_mean'] = (train['home_team_history_rating_1']+train['home_team_history_rating_2']+train['home_team_history_rating_3']+train['home_team_history_rating_4']+train['home_team_history_rating_5']+train['home_team_history_rating_6']+train['home_team_history_rating_7']+train['home_team_history_rating_8']+train['home_team_history_rating_9']+train['home_team_history_rating_10'])/10
train['away_team_history_rating_mean'] = (train['away_team_history_rating_1']+train['away_team_history_rating_2']+train['away_team_history_rating_3']+train['away_team_history_rating_4']+train['away_team_history_rating_5']+train['away_team_history_rating_6']+train['away_team_history_rating_7']+train['away_team_history_rating_8']+train['away_team_history_rating_9']+train['away_team_history_rating_10'])/10

In [50]:
train['home_team_history_opponent_rating_mean'] = (train['home_team_history_opponent_rating_1']+train['home_team_history_opponent_rating_2']+train['home_team_history_opponent_rating_3']+train['home_team_history_opponent_rating_4']+train['home_team_history_opponent_rating_5']+train['home_team_history_opponent_rating_6']+train['home_team_history_opponent_rating_7']+train['home_team_history_opponent_rating_8']+train['home_team_history_opponent_rating_9']+train['home_team_history_opponent_rating_10'])/10
train['away_team_history_opponent_rating_mean'] = (train['away_team_history_opponent_rating_1']+train['away_team_history_opponent_rating_2']+train['away_team_history_opponent_rating_3']+train['away_team_history_opponent_rating_4']+train['away_team_history_opponent_rating_5']+train['away_team_history_opponent_rating_6']+train['away_team_history_opponent_rating_7']+train['away_team_history_opponent_rating_8']+train['away_team_history_opponent_rating_9']+train['away_team_history_opponent_rating_10'])/10

In [53]:
train['saldo_goal_1'] = train['home_team_history_goal_1'] - train['home_team_history_opponent_goal_1']
train['saldo_goal_2'] = train['home_team_history_goal_2'] - train['home_team_history_opponent_goal_2']
train['saldo_goal_3'] = train['home_team_history_goal_3'] - train['home_team_history_opponent_goal_3']
train['saldo_goal_4'] = train['home_team_history_goal_4'] - train['home_team_history_opponent_goal_4']
train['saldo_goal_5'] = train['home_team_history_goal_5'] - train['home_team_history_opponent_goal_5']
train['saldo_goal_6'] = train['home_team_history_goal_6'] - train['home_team_history_opponent_goal_6']
train['saldo_goal_7'] = train['home_team_history_goal_7'] - train['home_team_history_opponent_goal_7']
train['saldo_goal_8'] = train['home_team_history_goal_8'] - train['home_team_history_opponent_goal_8']
train['saldo_goal_9'] = train['home_team_history_goal_9'] - train['home_team_history_opponent_goal_9']
train['saldo_goal_10'] = train['home_team_history_goal_10'] - train['home_team_history_opponent_goal_10']

train['saldo_goal_mean'] = (train['saldo_goal_1']+train['saldo_goal_2']+train['saldo_goal_3']+train['saldo_goal_4']+train['saldo_goal_5']+train['saldo_goal_6']+train['saldo_goal_7']+train['saldo_goal_8']+train['saldo_goal_9']+train['saldo_goal_10'])/10

In [54]:
train['saldo_rating_1'] = train['home_team_history_rating_1'] - train['home_team_history_opponent_rating_1']
train['saldo_rating_2'] = train['home_team_history_rating_2'] - train['home_team_history_opponent_rating_2']
train['saldo_rating_3'] = train['home_team_history_rating_3'] - train['home_team_history_opponent_rating_3']
train['saldo_rating_4'] = train['home_team_history_rating_4'] - train['home_team_history_opponent_rating_4']
train['saldo_rating_5'] = train['home_team_history_rating_5'] - train['home_team_history_opponent_rating_5']
train['saldo_rating_6'] = train['home_team_history_rating_6'] - train['home_team_history_opponent_rating_6']
train['saldo_rating_7'] = train['home_team_history_rating_7'] - train['home_team_history_opponent_rating_7']
train['saldo_rating_8'] = train['home_team_history_rating_8'] - train['home_team_history_opponent_rating_8']
train['saldo_rating_9'] = train['home_team_history_rating_9'] - train['home_team_history_opponent_rating_9']
train['saldo_rating_10'] = train['home_team_history_rating_10'] - train['home_team_history_opponent_rating_10']

train['saldo_rating_mean'] = (train['saldo_rating_1']+train['saldo_rating_2']+train['saldo_rating_3']+train['saldo_rating_4']+train['saldo_rating_5']+train['saldo_rating_6']+train['saldo_rating_7']+train['saldo_rating_8']+train['saldo_rating_9']+train['saldo_rating_10'])/10

In [61]:
def mudar_target(x):
    if x == 'away':
        return 0
    elif x == 'home':
        return 1
    elif x == 'draw':
        return 2
    else:
        return 99

train['target_'] = train['target'].apply(mudar_target)

train['target_'].value_counts()

1    48113
0    35174
2    27651
Name: target_, dtype: int64

In [55]:
features_names =[
    'home_team_history_is_play_home_perc',
    'away_team_history_is_play_home_perc',
    'home_team_history_is_cup_perc',
    'away_team_history_is_cup_perc',
    'home_team_history_goal_tot_perc',
    'away_team_history_goal_tot_perc',
    'home_team_history_goal_tot',
    'away_team_history_goal_tot',
    'home_team_history_opponent_goal_tot_perc',
    'away_team_history_opponent_goal_tot_perc',
    'home_team_history_opponent_goal_tot',
    'away_team_history_opponent_goal_tot',
    'home_team_history_rating_mean',
    'away_team_history_rating_mean',
    'home_team_history_opponent_rating_mean',
    'away_team_history_opponent_rating_mean',
    'saldo_goal_mean',
    'saldo_rating_mean'
]


In [58]:
X_train = train.set_index('id')[features_names].fillna(train.mean())
X_train.head()

,home_team_history_is_play_home_perc,away_team_history_is_play_home_perc,home_team_history_is_cup_perc,away_team_history_is_cup_perc,home_team_history_goal_tot_perc,away_team_history_goal_tot_perc,home_team_history_goal_tot,away_team_history_goal_tot,home_team_history_opponent_goal_tot_perc,away_team_history_opponent_goal_tot_perc,home_team_history_opponent_goal_tot,away_team_history_opponent_goal_tot,home_team_history_rating_mean,away_team_history_rating_mean,home_team_history_opponent_rating_mean,away_team_history_opponent_rating_mean,saldo_goal_mean,saldo_rating_mean
id,,,,,,,,,,,,,,,,,,
11906497,0.5,0.6,0.0,0.5,6,8,13.0,16.0,7,6,10.0,9.0,5.496371,8.561214,4.400204,3.808902,0.3,1.096167
11984383,0.5,0.5,0.0,0.0,4,5,9.0,8.0,7,7,10.0,14.0,8.440839,5.287316,4.546772,6.822376,-0.1,3.894067
11983301,0.5,0.5,0.0,0.0,9,7,19.0,21.0,8,6,17.0,11.0,6.844712,7.620071,5.547477,5.188232,0.2,1.297235
11983471,0.5,0.5,0.0,0.1,9,10,18.0,24.0,9,9,13.0,20.0,7.449923,5.818128,4.983015,6.469250,0.5,2.466908
11883005,0.5,0.6,0.0,0.0,10,8,18.0,18.0,9,7,14.0,10.0,7.079634,5.983338,5.192975,6.027979,0.4,1.886660


In [59]:
scaler = StandardScaler()
X = scaler.fit_transform(X_train)
X

array([[ 0.16034455,  0.87561779, -0.63784037, ..., -1.71615295,
         0.32436705,  0.40699728],
       [ 0.16034455, -0.19816636, -0.63784037, ...,  0.31784094,
        -0.18303093,  1.59928726],
       [ 0.16034455, -0.19816636, -0.63784037, ..., -0.78515185,
         0.19751755,  0.49268014],
       ...,
       [-0.92858718,  0.87561779,  0.94086659, ..., -1.36078903,
        -0.18303093,  0.42837594],
       [ 0.16034455,  0.87561779, -0.63784037, ..., -0.29252381,
        -0.56357941,  0.00770818],
       [ 0.16034455, -0.19816636,  0.15151311, ...,  0.18173183,
         0.57806604,  0.19873554]])

In [62]:
y = train['target_']
y

0         0
1         1
2         2
3         0
4         1
         ..
110933    2
110934    0
110935    2
110936    0
110937    1
Name: target_, Length: 110938, dtype: int64

In [63]:
clf = RandomForestClassifier(random_state=12345)
clf.fit(X,y)

RandomForestClassifier(random_state=12345)

In [64]:
pd.DataFrame(zip(X_train.columns, clf.feature_importances_),columns=['Variavel', 'feature_importances']).sort_values('feature_importances', ascending=False)

,Variavel,feature_importances
13,away_team_history_rating_mean,0.093565
15,away_team_history_opponent_rating_mean,0.090399
17,saldo_rating_mean,0.087514
12,home_team_history_rating_mean,0.083849
14,home_team_history_opponent_rating_mean,0.083233
7,away_team_history_goal_tot,0.056167
11,away_team_history_opponent_goal_tot,0.055766
16,saldo_goal_mean,0.055683
6,home_team_history_goal_tot,0.050142
10,home_team_history_opponent_goal_tot,0.049927


In [ ]:
#resultados = cross_val_predict(clf, X, y, cv = 5)
#print(metrics.accuracy_score(y, resultados))

## Predict with the first ML model

In [65]:
test = pd.read_csv('test.csv')
test.head()

,id,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_opponen

In [66]:
test['home_team_history_is_play_home_perc'] = (test['home_team_history_is_play_home_1']+test['home_team_history_is_play_home_2']+test['home_team_history_is_play_home_3']+test['home_team_history_is_play_home_4']+test['home_team_history_is_play_home_5']+test['home_team_history_is_play_home_6']+test['home_team_history_is_play_home_7']+test['home_team_history_is_play_home_8']+test['home_team_history_is_play_home_9']+test['home_team_history_is_play_home_10']) / 10
test['away_team_history_is_play_home_perc'] = (test['away_team_history_is_play_home_1']+test['away_team_history_is_play_home_2']+test['away_team_history_is_play_home_3']+test['away_team_history_is_play_home_4']+test['away_team_history_is_play_home_5']+test['away_team_history_is_play_home_6']+test['away_team_history_is_play_home_7']+test['away_team_history_is_play_home_8']+test['away_team_history_is_play_home_9']+test['away_team_history_is_play_home_10']) / 10

In [67]:
test['home_team_history_is_cup_perc'] = (test['home_team_history_is_cup_1']+test['home_team_history_is_cup_2']+test['home_team_history_is_cup_3']+test['home_team_history_is_cup_4']+test['home_team_history_is_cup_5']+test['home_team_history_is_cup_6']+test['home_team_history_is_cup_7']+test['home_team_history_is_cup_8']+test['home_team_history_is_cup_9']+test['home_team_history_is_cup_10']) / 10
test['away_team_history_is_cup_perc'] = (test['away_team_history_is_cup_1']+test['away_team_history_is_cup_2']+test['away_team_history_is_cup_3']+test['away_team_history_is_cup_4']+test['away_team_history_is_cup_5']+test['away_team_history_is_cup_6']+test['away_team_history_is_cup_7']+test['away_team_history_is_cup_8']+test['away_team_history_is_cup_9']+test['away_team_history_is_cup_10']) / 10

In [68]:
test['home_team_history_goal_1_ind'] = test['home_team_history_goal_1'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_2_ind'] = test['home_team_history_goal_2'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_3_ind'] = test['home_team_history_goal_3'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_4_ind'] = test['home_team_history_goal_4'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_5_ind'] = test['home_team_history_goal_5'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_6_ind'] = test['home_team_history_goal_6'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_7_ind'] = test['home_team_history_goal_7'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_8_ind'] = test['home_team_history_goal_8'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_9_ind'] = test['home_team_history_goal_9'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_goal_10_ind'] = test['home_team_history_goal_10'].apply(lambda x: 1 if x > 0 else 0)

test['home_team_history_goal_tot_perc'] = (test['home_team_history_goal_1_ind']+test['home_team_history_goal_2_ind']+test['home_team_history_goal_3_ind']+test['home_team_history_goal_4_ind']+test['home_team_history_goal_5_ind']+test['home_team_history_goal_6_ind']+test['home_team_history_goal_7_ind']+test['home_team_history_goal_8_ind']+test['home_team_history_goal_9_ind']+test['home_team_history_goal_10_ind'])



test['away_team_history_goal_1_ind'] = test['away_team_history_goal_1'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_2_ind'] = test['away_team_history_goal_2'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_3_ind'] = test['away_team_history_goal_3'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_4_ind'] = test['away_team_history_goal_4'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_5_ind'] = test['away_team_history_goal_5'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_6_ind'] = test['away_team_history_goal_6'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_7_ind'] = test['away_team_history_goal_7'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_8_ind'] = test['away_team_history_goal_8'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_9_ind'] = test['away_team_history_goal_9'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_goal_10_ind'] = test['away_team_history_goal_10'].apply(lambda x: 1 if x > 0 else 0)

test['away_team_history_goal_tot_perc'] = (test['away_team_history_goal_1_ind']+test['away_team_history_goal_2_ind']+test['away_team_history_goal_3_ind']+test['away_team_history_goal_4_ind']+test['away_team_history_goal_5_ind']+test['away_team_history_goal_6_ind']+test['away_team_history_goal_7_ind']+test['away_team_history_goal_8_ind']+test['away_team_history_goal_9_ind']+test['away_team_history_goal_10_ind'])


test['home_team_history_goal_tot'] = (test['home_team_history_goal_1']+test['home_team_history_goal_2']+test['home_team_history_goal_3']+test['home_team_history_goal_4']+test['home_team_history_goal_5']+test['home_team_history_goal_6']+test['home_team_history_goal_7']+test['home_team_history_goal_8']+test['home_team_history_goal_9']+test['home_team_history_goal_10'])
test['away_team_history_goal_tot'] = (test['away_team_history_goal_1']+test['away_team_history_goal_2']+test['away_team_history_goal_3']+test['away_team_history_goal_4']+test['away_team_history_goal_5']+test['away_team_history_goal_6']+test['away_team_history_goal_7']+test['away_team_history_goal_8']+test['away_team_history_goal_9']+test['away_team_history_goal_10'])

In [69]:
test['home_team_history_opponent_goal_1_ind'] = test['home_team_history_opponent_goal_1'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_2_ind'] = test['home_team_history_opponent_goal_2'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_3_ind'] = test['home_team_history_opponent_goal_3'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_4_ind'] = test['home_team_history_opponent_goal_4'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_5_ind'] = test['home_team_history_opponent_goal_5'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_6_ind'] = test['home_team_history_opponent_goal_6'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_7_ind'] = test['home_team_history_opponent_goal_7'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_8_ind'] = test['home_team_history_opponent_goal_8'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_9_ind'] = test['home_team_history_opponent_goal_9'].apply(lambda x: 1 if x > 0 else 0)
test['home_team_history_opponent_goal_10_ind'] = test['home_team_history_opponent_goal_10'].apply(lambda x: 1 if x > 0 else 0)

test['home_team_history_opponent_goal_tot_perc'] = (test['home_team_history_opponent_goal_1_ind']+test['home_team_history_opponent_goal_2_ind']+test['home_team_history_opponent_goal_3_ind']+test['home_team_history_opponent_goal_4_ind']+test['home_team_history_opponent_goal_5_ind']+test['home_team_history_opponent_goal_6_ind']+test['home_team_history_opponent_goal_7_ind']+test['home_team_history_opponent_goal_8_ind']+test['home_team_history_opponent_goal_9_ind']+test['home_team_history_opponent_goal_10_ind'])



test['away_team_history_opponent_goal_1_ind'] = test['away_team_history_opponent_goal_1'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_2_ind'] = test['away_team_history_opponent_goal_2'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_3_ind'] = test['away_team_history_opponent_goal_3'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_4_ind'] = test['away_team_history_opponent_goal_4'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_5_ind'] = test['away_team_history_opponent_goal_5'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_6_ind'] = test['away_team_history_opponent_goal_6'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_7_ind'] = test['away_team_history_opponent_goal_7'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_8_ind'] = test['away_team_history_opponent_goal_8'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_9_ind'] = test['away_team_history_opponent_goal_9'].apply(lambda x: 1 if x > 0 else 0)
test['away_team_history_opponent_goal_10_ind'] = test['away_team_history_opponent_goal_10'].apply(lambda x: 1 if x > 0 else 0)

test['away_team_history_opponent_goal_tot_perc'] = (test['away_team_history_opponent_goal_1_ind']+test['away_team_history_opponent_goal_2_ind']+test['away_team_history_opponent_goal_3_ind']+test['away_team_history_opponent_goal_4_ind']+test['away_team_history_opponent_goal_5_ind']+test['away_team_history_opponent_goal_6_ind']+test['away_team_history_opponent_goal_7_ind']+test['away_team_history_opponent_goal_8_ind']+test['away_team_history_opponent_goal_9_ind']+test['away_team_history_opponent_goal_10_ind'])


test['home_team_history_opponent_goal_tot'] = (test['home_team_history_opponent_goal_1']+test['home_team_history_opponent_goal_2']+test['home_team_history_opponent_goal_3']+test['home_team_history_opponent_goal_4']+test['home_team_history_opponent_goal_5']+test['home_team_history_opponent_goal_6']+test['home_team_history_opponent_goal_7']+test['home_team_history_opponent_goal_8']+test['home_team_history_opponent_goal_9']+test['home_team_history_opponent_goal_10'])
test['away_team_history_opponent_goal_tot'] = (test['away_team_history_opponent_goal_1']+test['away_team_history_opponent_goal_2']+test['away_team_history_opponent_goal_3']+test['away_team_history_opponent_goal_4']+test['away_team_history_opponent_goal_5']+test['away_team_history_opponent_goal_6']+test['away_team_history_opponent_goal_7']+test['away_team_history_opponent_goal_8']+test['away_team_history_opponent_goal_9']+test['away_team_history_opponent_goal_10'])

In [70]:
test['home_team_history_rating_mean'] = (test['home_team_history_rating_1']+test['home_team_history_rating_2']+test['home_team_history_rating_3']+test['home_team_history_rating_4']+test['home_team_history_rating_5']+test['home_team_history_rating_6']+test['home_team_history_rating_7']+test['home_team_history_rating_8']+test['home_team_history_rating_9']+test['home_team_history_rating_10'])/10
test['away_team_history_rating_mean'] = (test['away_team_history_rating_1']+test['away_team_history_rating_2']+test['away_team_history_rating_3']+test['away_team_history_rating_4']+test['away_team_history_rating_5']+test['away_team_history_rating_6']+test['away_team_history_rating_7']+test['away_team_history_rating_8']+test['away_team_history_rating_9']+test['away_team_history_rating_10'])/10

In [71]:
test['home_team_history_opponent_rating_mean'] = (test['home_team_history_opponent_rating_1']+test['home_team_history_opponent_rating_2']+test['home_team_history_opponent_rating_3']+test['home_team_history_opponent_rating_4']+test['home_team_history_opponent_rating_5']+test['home_team_history_opponent_rating_6']+test['home_team_history_opponent_rating_7']+test['home_team_history_opponent_rating_8']+test['home_team_history_opponent_rating_9']+test['home_team_history_opponent_rating_10'])/10
test['away_team_history_opponent_rating_mean'] = (test['away_team_history_opponent_rating_1']+test['away_team_history_opponent_rating_2']+test['away_team_history_opponent_rating_3']+test['away_team_history_opponent_rating_4']+test['away_team_history_opponent_rating_5']+test['away_team_history_opponent_rating_6']+test['away_team_history_opponent_rating_7']+test['away_team_history_opponent_rating_8']+test['away_team_history_opponent_rating_9']+test['away_team_history_opponent_rating_10'])/10

In [72]:
test['saldo_goal_1'] = test['home_team_history_goal_1'] - test['home_team_history_opponent_goal_1']
test['saldo_goal_2'] = test['home_team_history_goal_2'] - test['home_team_history_opponent_goal_2']
test['saldo_goal_3'] = test['home_team_history_goal_3'] - test['home_team_history_opponent_goal_3']
test['saldo_goal_4'] = test['home_team_history_goal_4'] - test['home_team_history_opponent_goal_4']
test['saldo_goal_5'] = test['home_team_history_goal_5'] - test['home_team_history_opponent_goal_5']
test['saldo_goal_6'] = test['home_team_history_goal_6'] - test['home_team_history_opponent_goal_6']
test['saldo_goal_7'] = test['home_team_history_goal_7'] - test['home_team_history_opponent_goal_7']
test['saldo_goal_8'] = test['home_team_history_goal_8'] - test['home_team_history_opponent_goal_8']
test['saldo_goal_9'] = test['home_team_history_goal_9'] - test['home_team_history_opponent_goal_9']
test['saldo_goal_10'] = test['home_team_history_goal_10'] - test['home_team_history_opponent_goal_10']

test['saldo_goal_mean'] = (test['saldo_goal_1']+test['saldo_goal_2']+test['saldo_goal_3']+test['saldo_goal_4']+test['saldo_goal_5']+test['saldo_goal_6']+test['saldo_goal_7']+test['saldo_goal_8']+test['saldo_goal_9']+test['saldo_goal_10'])/10

In [73]:
test['saldo_rating_1'] = test['home_team_history_rating_1'] - test['home_team_history_opponent_rating_1']
test['saldo_rating_2'] = test['home_team_history_rating_2'] - test['home_team_history_opponent_rating_2']
test['saldo_rating_3'] = test['home_team_history_rating_3'] - test['home_team_history_opponent_rating_3']
test['saldo_rating_4'] = test['home_team_history_rating_4'] - test['home_team_history_opponent_rating_4']
test['saldo_rating_5'] = test['home_team_history_rating_5'] - test['home_team_history_opponent_rating_5']
test['saldo_rating_6'] = test['home_team_history_rating_6'] - test['home_team_history_opponent_rating_6']
test['saldo_rating_7'] = test['home_team_history_rating_7'] - test['home_team_history_opponent_rating_7']
test['saldo_rating_8'] = test['home_team_history_rating_8'] - test['home_team_history_opponent_rating_8']
test['saldo_rating_9'] = test['home_team_history_rating_9'] - test['home_team_history_opponent_rating_9']
test['saldo_rating_10'] = test['home_team_history_rating_10'] - test['home_team_history_opponent_rating_10']

test['saldo_rating_mean'] = (test['saldo_rating_1']+test['saldo_rating_2']+test['saldo_rating_3']+test['saldo_rating_4']+test['saldo_rating_5']+test['saldo_rating_6']+test['saldo_rating_7']+test['saldo_rating_8']+test['saldo_rating_9']+test['saldo_rating_10'])/10

In [74]:
test_ = test.set_index('id')[features_names].fillna(test.mean())
test_.head()

,home_team_history_is_play_home_perc,away_team_history_is_play_home_perc,home_team_history_is_cup_perc,away_team_history_is_cup_perc,home_team_history_goal_tot_perc,away_team_history_goal_tot_perc,home_team_history_goal_tot,away_team_history_goal_tot,home_team_history_opponent_goal_tot_perc,away_team_history_opponent_goal_tot_perc,home_team_history_opponent_goal_tot,away_team_history_opponent_goal_tot,home_team_history_rating_mean,away_team_history_rating_mean,home_team_history_opponent_rating_mean,away_team_history_opponent_rating_mean,saldo_goal_mean,saldo_rating_mean
id,,,,,,,,,,,,,,,,,,
17761448,0.4,0.5,0.4,0.0,6,5,10.0,11.0,4,7,6.0,14.0,4.570763,4.811870,7.408696,6.221093,0.4,-2.837933
17695487,0.5,0.6,0.0,0.0,6,6,7.0,10.0,8,5,14.0,10.0,5.203757,5.885441,6.435353,5.629701,-0.7,-1.231596
17715496,0.5,0.6,0.0,0.0,5,4,8.0,6.0,9,9,14.0,15.0,4.474860,6.737222,6.684485,6.559763,-0.6,-2.209625
17715493,0.4,0.6,0.0,0.2,6,8,8.0,11.0,8,5,16.0,8.0,4.561036,6.809434,6.706340,4.405098,-0.8,-2.145304
17715492,0.4,0.5,0.0,0.0,7,4,9.0,6.0,8,7,12.0,9.0,5.102046,4.377685,5.986625,6.635796,-0.3,-0.884579


In [75]:
X_test = scaler.fit_transform(test_)
X_test

array([[-8.68457852e-01, -1.80139285e-01,  2.09747988e+00, ...,
        -1.95341936e-01,  4.13547353e-01, -1.15938753e+00],
       [ 1.57694270e-01,  8.38839244e-01, -5.46261997e-01, ...,
        -5.36452394e-01, -8.57822968e-01, -5.33268402e-01],
       [ 1.57694270e-01,  8.38839244e-01, -5.46261997e-01, ...,
         5.12294097e-16, -7.42243848e-01, -9.14485108e-01],
       ...,
       [ 1.18384639e+00, -1.80139285e-01,  1.14673472e-01, ...,
        -4.23174403e-01, -7.42243848e-01, -1.85804804e+00],
       [-8.68457852e-01, -1.80139285e-01,  1.14673472e-01, ...,
        -8.45363505e-01,  1.33818031e+00,  1.19014911e+00],
       [ 1.57694270e-01, -1.80139285e-01, -5.46261997e-01, ...,
        -1.01994528e+00, -1.64348247e-01,  2.55320981e-01]])

In [80]:
preds = clf.predict_proba(X_test)
preds

array([[0.17, 0.37, 0.46],
       [0.37, 0.43, 0.2 ],
       [0.27, 0.38, 0.35],
       ...,
       [0.63, 0.1 , 0.27],
       [0.24, 0.43, 0.33],
       [0.38, 0.35, 0.27]])

In [82]:
preds_ = pd.DataFrame(preds, columns=['away','home','draw'])
preds_.head()

,away,home,draw
0,0.17,0.37,0.46
1,0.37,0.43,0.20
2,0.27,0.38,0.35
3,0.37,0.29,0.34
4,0.25,0.51,0.24


In [78]:
test_ = test_.reset_index(level=0)
test_.head()

,id,home_team_history_is_play_home_perc,away_team_history_is_play_home_perc,home_team_history_is_cup_perc,away_team_history_is_cup_perc,home_team_history_goal_tot_perc,away_team_history_goal_tot_perc,home_team_history_goal_tot,away_team_history_goal_tot,home_team_history_opponent_goal_tot_perc,away_team_history_opponent_goal_tot_perc,home_team_history_opponent_goal_tot,away_team_history_opponent_goal_tot,home_team_history_rating_mean,away_team_history_rating_mean,home_team_history_opponent_rating_mean,away_team_history_opponent_rating_mean,saldo_goal_mean,saldo_rating_mean
0,17761448,0.4,0.5,0.4,0.0,6,5,10.0,11.0,4,7,6.0,14.0,4.570763,4.811870,7.408696,6.221093,0.4,-2.837933
1,17695487,0.5,0.6,0.0,0.0,6,6,7.0,10.0,8,5,14.0,10.0,5.203757,5.885441,6.435353,5.629701,-0.7,-1.231596
2,17715496,0.5,0.6,0.0,0.0,5,4,8.0,6.0,9,9,14.0,15.0,4.474860,6.737222,6.684485,6.559763,-0.6,-2.209625
3,17715493,0.4,0.6,0.0,0.2,6,8,8.0,11.0,8,5,16.0,8.0,4.561036,6.809434,6.706340,4.405098,-0.8,-2.145304
4,17715492,0.4,0.5,0.0,0.0,7,4,9.0,6.0,8,7,12.0,9.0,5.102046,4.377685,5.986625,6.635796,-0.3,-0.884579


In [83]:
sub01 = pd.concat([test_['id'],preds_], axis=1)
sub01.head()

,id,away,home,draw
0,17761448,0.17,0.37,0.46
1,17695487,0.37,0.43,0.20
2,17715496,0.27,0.38,0.35
3,17715493,0.37,0.29,0.34
4,17715492,0.25,0.51,0.24


In [84]:
feature_target =[
    'id',
    'home',
    'draw',
    'away'
]

In [85]:
sub01_ = sub01[feature_target]
sub01_.head()

,id,home,draw,away
0,17761448,0.37,0.46,0.17
1,17695487,0.43,0.20,0.37
2,17715496,0.38,0.35,0.27
3,17715493,0.29,0.34,0.37
4,17715492,0.51,0.24,0.25


In [86]:
sub01_.to_csv('sub01.csv', sep=',', index=False)

In [31]:
train['is_cup'].value_counts()

False    101856
True       9081
Name: is_cup, dtype: int64

In [7]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission.head()

,id,home,draw,away
0,17761448,0.333,0.333,0.333
1,17695487,0.333,0.333,0.333
2,17715496,0.333,0.333,0.333
3,17715493,0.333,0.333,0.333
4,17715492,0.333,0.333,0.333
